In [2]:
# ! pip install datasets transformers sacrebleu accelerate datasets

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

# %cd drive/MyDrive/text-detoxification/notebooks/

In [4]:
import pandas as pd
import numpy as np
import datasets
import warnings
warnings.filterwarnings("ignore")

In [5]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_metric, Dataset

In [6]:
# load the datasets
train = pd.read_csv("../data/interim/train.csv")
test = pd.read_csv("../data/interim/test.csv")

In [7]:
datasets = datasets.DatasetDict({"train":Dataset.from_pandas(train),"test":Dataset.from_pandas(test)})

In [8]:
tokenizer = AutoTokenizer.from_pretrained('t5-base')

#### get tokens from reference and translated sentences

In [9]:
def preprocess_function(dataset):
    model_inputs = tokenizer(dataset["reference"], truncation=True)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(dataset["translation"], truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [10]:
tokenized_datasets = datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

In [11]:
model = AutoModelForSeq2SeqLM.from_pretrained('t5-base')
model.save_pretrained("../models/t5-base/")

batch_size = 4
args = Seq2SeqTrainingArguments(
    f"../models/t5-base/",
    evaluation_strategy = "epoch",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    fp16=True,
    num_train_epochs=1,
    save_steps=10000
)

In [12]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

#### define metrics calculators

In [13]:
metric = load_metric("sacrebleu")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [14]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [15]:
trainer.train()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss


OutOfMemoryError: ignored

In [ ]:
model.save_pretrained("../models/t5-base/pretrained")

In [ ]:
text = "oh, I have fucked up, sorry. what can I do for you, fucking nerd?"
tokenized_text = tokenizer(text, return_tensors = "pt")


In [ ]:
out = model.generate(input_ids = tokenized_text["input_ids"], attention_mask = tokenized_text["attention_mask"], max_length = 128, num_return_sequences = 1,)
preds = [
        tokenizer.decode(gen_id,
        skip_special_tokens = True,
        clean_up_tokenization_spaces=True)
        for gen_id in out]
preds[0]